# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [3]:
import pandas as pd

## imports for Python, Pandas

In [5]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [4]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [7]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [8]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [1]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [10]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

#### Solution:
*NOTE: I will assume that the previous cells of the notebook have all been executed and will not re-import pandas.*

First I set the filepath for '`world_bank_projects.json`' to the string '`path`'. Then read it into a pandas dataframe '`bankDF`'.

I examined the contents of the dataframe with the .info() method, but commented it out in the submission for easier viewing.

In [4]:
path = r'C:\Users\glander\Documents\DataScience\Springboard\Assessments\data_wrangling_json\data_wrangling_json\data\world_bank_projects.json'
bankDF = pd.read_json(path)

# bankDF.info()

**1. Find the 10 countries with the most projects**

Since each observation in the dataframe corresponds to a project, the countries with the most projects can be found with the value_counts() method used on the 'countryname' feature.

In [14]:
bankDF.countryname.value_counts()[:10]

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

**2. Find the top 10 major project themes (using column 'mjtheme_namecode')**

I'll extract the json information from the mjtheme_namecode column of `bankDF` and form a separate dataframe with its contents. To keep track of where each code-name combination from the json field is coming from, I'll carry over an index-like column from `bankDF`. The project_name column looks like a unique identifier, and a quick test below confirms this.

In [92]:
bankDF.project_name.unique().shape == bankDF.project_name.shape

True

Next, I'll form a new dataframe, `codeDF` by concatenating a dictionary comprehension of each observation, where project_name is used as the key and then becomes the index of `codeDF` and the output of json_normalize creates and populates two columns, code and name.

Because the concatenation creates a multi-level index (since each code is interpretted as a separate observation), I do a little rejiggering below to make project_name a proper feature of `codeDF` and get rid of the second index level.

In [89]:
codeDF = pd.concat({p: json_normalize(row) for p,row in zip(bankDF.project_name, bankDF.mjtheme_namecode)}, axis = 'rows')

codeDF.reset_index(inplace = True)
print(codeDF.head(2),'\n')
codeDF.rename(columns={"level_0": "project_name"},inplace=True)
codeDF.drop(['level_1'],axis='columns',inplace=True)
print('After adjustments:', codeDF.head(2))

                                    level_0  level_1 code  \
0  5M: Displaced People in Jordan / Lebanon        0    2   
1  5M: Displaced People in Jordan / Lebanon        1    6   

                                    name  
0               Public sector governance  
1  Social protection and risk management   

After adjustments:                                project_name code  \
0  5M: Displaced People in Jordan / Lebanon    2   
1  5M: Displaced People in Jordan / Lebanon    6   

                                    name  
0               Public sector governance  
1  Social protection and risk management  


Now all that's left is to group `codeDF` by code and count the results of each code. The total for each code is stored as a  feature, and the top ten code frequencies are displayed below.

In [90]:
codeDF['total'] = codeDF.groupby('code')['code'].transform('count')

codeDF.loc[codeDF.name != '', ['code','name','total']].drop_duplicates().sort_values(by='total', ascending = False)[:10]

,code,name,total
4,11,Environment and natural resources management,250
7,10,Rural development,216
61,8,Human development,210
0,2,Public sector governance,199
1,6,Social protection and risk management,168
26,4,Financial and private sector development,146
9,7,Social dev/gender/inclusion,130
14,5,Trade and integration,77
152,9,Urban development,50
21,1,Economic management,38


**3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.**

*NOTE: I'm a little confused by this instruction - Is it asking me to create an updated version of the original dataframe with the corrected names filled into the json field (mjtheme_namecode), or a clean lookup table type dataframe with the codes and names all correctly listed? I opted to do the latter.*

First, I converted all the code-name pairs into a dictionary using a dict comprehension.

In [94]:
short_codeDF = codeDF[codeDF.name != ''].drop_duplicates(['code', 'name'])
code_name_dict = {code: name for code, name in zip(short_codeDF.code, short_codeDF.name)}
print(code_name_dict)

{'2': 'Public sector governance', '6': 'Social protection and risk management', '11': 'Environment and natural resources management', '10': 'Rural development', '7': 'Social dev/gender/inclusion', '5': 'Trade and integration', '1': 'Economic management', '4': 'Financial and private sector development', '8': 'Human development', '9': 'Urban development', '3': 'Rule of law'}


Then I used code_name_dict to map the proper name to all code values. I printed the first 5 missing rows before and after to confirm the operation was successful.

Finally, because I have preserved the project_name field of the original dataframe in this lookup dataframe, I can access this information in the original dataframe if needed.

In [115]:
missing = codeDF.name == ''
print(codeDF[missing].head(5))
codeDF.name = codeDF.code.map(code_name_dict)
print(codeDF[missing].head(5))

                                          project_name code name  total
57   Additional Financing Belarus Energy Efficiency...   10         216
64   Additional Financing for Cash Transfers for Or...    6         168
137  An Innovative, Integrated Approach to Enhance ...   11         250
139                    Angola Learning for All Project    4         146
141  Anhui Xuancheng Infrastructure for Industry Re...    2         199
                                          project_name code  \
57   Additional Financing Belarus Energy Efficiency...   10   
64   Additional Financing for Cash Transfers for Or...    6   
137  An Innovative, Integrated Approach to Enhance ...   11   
139                    Angola Learning for All Project    4   
141  Anhui Xuancheng Infrastructure for Industry Re...    2   

                                             name  total  
57                              Rural development    216  
64          Social protection and risk management    168  
137  Environ